In [13]:
from typing import List, Tuple, Dict

def unify(term1: str, term2: str) -> Dict[str, str]:
    """
    Unify two terms. If unification is possible, return a substitution dictionary.
    Otherwise, return None.
    """
    if term1 == term2:
        return {}
    elif term1.startswith('~') and term2.startswith('~'):
        return unify(term1[1:], term2[1:])
    elif term1.startswith('~'):
        return unify(term1[1:], term2)
    elif term2.startswith('~'):
        return unify(term1, term2[1:])
    elif term1[0].isupper() and term2[0].islower():
        return {term2: term1}
    elif term2[0].isupper() and term1[0].islower():
        return {term1: term2}
    return None

def substitute_literal(literal: str, substitution: Dict[str, str]) -> str:
    """
    Substitute variables in a literal based on the given substitution dictionary.
    """
    for var, value in substitution.items():
        if var in literal:
            literal = literal.replace(var, value)
    return literal

def resolve_clause(clause1: set, clause2: set) -> Tuple[bool, set]:
    """
    Attempt to resolve two clauses by unifying complementary literals.
    """
    for literal1 in clause1:
        for literal2 in clause2:
            unified = unify(literal1, literal2)
            if unified is not None:
                # Resolve the clauses by removing complementary literals and applying substitutions
                new_clause = (clause1 | clause2) - {literal1, literal2}
                resolved_clause = {substitute_literal(lit, unified) for lit in new_clause}
                return True, resolved_clause
    return False, set()

def perform_resolution(knowledge_base: List[set], target: set) -> Tuple[bool, List[Tuple[set, List[int]]]]:
    """
    Perform resolution and build the proof tree with optimizations.
    """
    proof_path = []
    clauses_set = knowledge_base + [target]  # Add negated target to the knowledge base
    new_clauses_set = []
    processed_clauses = set()  # To track already processed clauses and avoid redundant work

    while True:
        clause_pairs = [(clauses_set[i], clauses_set[j], (i, j))
                        for i in range(len(clauses_set)) for j in range(i + 1, len(clauses_set))]

        for clause_a, clause_b, (i, j) in clause_pairs:
            # Skip pairs that have already been processed
            if (i, j) in processed_clauses:
                continue
            processed_clauses.add((i, j))

            complementary, resolved = resolve_clause(clause_a, clause_b)
            if complementary:
                if not resolved:  # Empty clause found
                    proof_path.append((resolved, [i, j]))
                    return True, proof_path
                if resolved not in clauses_set and resolved not in new_clauses_set:
                    new_clauses_set.append(resolved)
                    proof_path.append((resolved, [i, j]))

        if not new_clauses_set:  # No progress, resolution failed
            return False, proof_path

        # Add only the new clauses that bring us closer to the goal
        clauses_set.extend(new_clauses_set)
        new_clauses_set = []

# Knowledge Base and Goal
knowledge_base = [
    {"Mother(Leela, Oshin)"},         # Clause 1: Mother(Leela, Oshin)
    {"Alive(Leela)"},                 # Clause 2: Alive(Leela)
    {"~Mother(x, y)", "Parent(x, y)"},  # Clause 3: ~Mother(x, y) or Parent(x, y)
    {"~Parent(x, y)", "~Alive(x)", "Older(x, y)"}  # Clause 4: ~Parent(x, y) or ~Alive(x) or Older(x, y)
]

negated_goal = {"~Older(Leela, Oshin)"}  # Negated Goal: ~Older(Leela, Oshin)

# Perform resolution
result, proof_path = perform_resolution(knowledge_base, negated_goal)

if result:
    print("PROVED")
else:
    print("CAN'T BE PROVED")

print("\nProof Path:")
for step, (clause, origins) in enumerate(proof_path):
    print(f"Step {step + 1}: {clause} derived from {origins}")


PROVED

Proof Path:
Step 1: {'~Alive(x)', 'Older(x, y)', '~Mother(x, y)'} derived from [2, 3]
Step 2: {'~Alive(x)', 'Older(x, y)', 'Parent(x, y)'} derived from [2, 5]
Step 3: {'~Alive(x)', '~Parent(x, y)', '~Mother(x, y)'} derived from [3, 5]
Step 4: {'~Alive(x)', '~Parent(x, y)', 'Parent(x, y)'} derived from [2, 7]
Step 5: {'~Alive(x)', 'Older(x, y)'} derived from [3, 6]
Step 6: {'~Mother(x, y)', 'Older(x, y)', 'Parent(x, y)'} derived from [5, 6]
Step 7: {'~Mother(x, y)', '~Parent(x, y)', 'Older(x, y)'} derived from [5, 7]
Step 8: {'~Parent(x, y)', 'Older(x, y)', '~Mother(x, y)', 'Parent(x, y)'} derived from [6, 7]
Step 9: {'~Alive(x)', '~Mother(x, y)'} derived from [2, 8]
Step 10: {'Older(x, y)', 'Parent(x, y)'} derived from [2, 10]
Step 11: {'~Parent(x, y)', 'Older(x, y)', 'Parent(x, y)'} derived from [2, 11]
Step 12: {'~Alive(x)', '~Parent(x, y)'} derived from [3, 9]
Step 13: {'~Alive(x)', 'Older(x, y)', '~Mother(x, y)', 'Parent(x, y)'} derived from [3, 12]
Step 14: {'~Mother(x, y)